## Cargar variables de entorno

In [1]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=".env", override=True)

True

## Crear aplicación AI 

### Setup 

Como siempre, definamos nuestro prompt y demos a nuestra aplicación acceso a la web.

In [2]:
# Inicializar herramienta de búsqueda web.
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

# Definir prompt template
prompt = """Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma que incluso un niño de 5 años pueda comprenderla.
Se te ha brindado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:"""

C:\Users\sergi\AppData\Local\Temp\ipykernel_55712\750042844.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(max_results=1)


### Definir la lógica de la aplicación.

La lógica acá es la misma que en el módulo de trazas. Definimos un paso de búsqueda para explorar la web y un paso de explicación para que un modelo de lenguaje resuma los resultados encontrados.

In [ ]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai


# Crear application
openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    formatted = prompt.format(question=question, context=context)
    
    completion = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": formatted},
            {"role": "user", "content": question},
        ],
        model="o3-mini",
    )
    return completion.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer


## Setup del experimento

Ahora estamos listos para ejecutar experimentos y probar el rendimiento de nuestra aplicación sobre nuestro dataset.

### Importar cliente LangSmith 

Primero, vamos a crear un cliente de LangSmith para usar el SDK y especificar el dataset sobre el que queremos ejecutar nuestro experimento.

In [4]:
from langsmith import Client

client = Client()
#dataset_name = "eli5-silver"
dataset_name = "ds-new-crystallography-60"

### Definir evaluadores

#### Evaluador de código personalizado

Primero definiremos un evaluador de código personalizado, que resulta útil para medir métricas deterministas o de respuesta cerrada.

In [5]:
def conciseness(outputs: dict) -> bool:
    words = outputs["output"].split(" ")
    return len(words) <= 200

Este evaluador de código personalizado es simplemente una función de Python que verifica si nuestra aplicación produce respuestas de 200 palabras o menos.

#### LLM-as-a-Judge Evaluador

Para métricas abiertas, puede ser muy potente usar un LLM para puntuar las respuestas.

Usemos un LLM para comprobar si nuestra aplicación produce resultados correctos. Primero, definamos un esquema de puntuación que nuestro LLM deba seguir en su respuesta.

In [6]:
from pydantic import BaseModel, Field

# Definir un esquema de puntuación al que nuestro LLM debe ajustarse.
class CorrectnessScore(BaseModel):
    """Correctness score of the answer when compared to the reference answer."""
    score: int = Field(description="The score of the correctness of the answer, from 0 to 1")

Vamos a definir una función para darle a un LLM las salidas de nuestra aplicación, junto con las salidas de referencia guardadas en nuestro conjunto de datos.

De este modo, el LLM podrá usar la respuesta “correcta” como referencia para juzgar si la respuesta de nuestra aplicación cumple con nuestros estándares de precisión.

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    prompt = """
    Sos un etiquetador de datos experto que evalúa las respuestas de un modelo para verificar su corrección.
Tu tarea es asignar una puntuación basada en la siguiente rúbrica:

    <Rubric>
        Una respuesta correcta:
            - Brinda información precisa
            - Usa analogías y ejemplos adecuados
            - No contiene errores fácticos
            - Es lógicamente consistente

        Al puntuar, debés penalizar:
            - Errores fácticos
            - Analogías y ejemplos incoherentes
            - Inconsistencias lógicas    
    </Rubric>

    <Instructions>
        - Leé con atención la entrada y la salida.
        - Usá la salida de referencia para determinar si la salida del modelo contiene errores.
        - Concentrate en si la salida del modelo usa analogías precisas y es lógicamente consistente.
    </Instructions>

    <Reminder>
        Las analogías de la salida no necesitan coincidir exactamente con la salida de referencia. Concentrate en la consistencia lógica.
    </Reminder>

    <input>
        {}
    </input>

    <output>
        {}
    </output>

    Usá las salidas de referencia que aparecen abajo para ayudarte a evaluar la corrección de la respuesta.
    <reference_outputs>
        {}
    </reference_outputs>
    """.format(inputs["question"], outputs["output"], reference_outputs["output"])
    structured_llm = ChatOpenAI(model_name="gpt-4o", temperature=0).with_structured_output(CorrectnessScore)
    generation = structured_llm.invoke([HumanMessage(content=prompt)])
    return generation.score == 1


### Definir Run Function

Vamos a definir una función para ejecutar nuestra aplicación sobre las entradas de ejemplo de nuestro dataset. Esta es la función que se va a llamar cuando ejecutemos nuestro experimento.

In [8]:
# Definir una función para ejecutar tu aplicación.
def run(inputs: dict):
    return eli5(inputs["question"])

## Correr experimento

Tenemos todos los componentes necesarios, así que ejecutemos nuestro experimento!

In [ ]:
from langsmith import evaluate

evaluate(
    run,
    data=dataset_name,
    evaluators=[correctness, conciseness],
    experiment_prefix="eli5-o3-mini"
)

c:\Users\sergi\Documents\langsmith\eli5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'eli5-o4-mini-e36162a4' at:
https://smith.langchain.com/o/11afda74-8804-4cb8-8ad4-c2a9d67d44e5/datasets/0a5a0302-cfaf-44b4-9de9-e4b312e514fd/compare?selectedSessions=3af3ae00-128f-46f1-9e99-55d2e97544d5




0it [00:00, ?it/s]c:\Users\sergi\Documents\langsmith\eli5\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
10it [01:20,  8.04s/it]


,inputs.question,outputs.output,error,reference.output,feedback.correctness,feedback.conciseness,execution_time,example_id,id
0,Qué es la macroeconomía?,Imagina que la economía de un país es como una...,None,La macroeconomía es como una lupa gigante que ...,True,True,5.993207,15d24932-295f-473e-b1d2-086f1bd2f32d,019a9efa-135a-7140-a800-9171ae7f4598
1,Why is the sky blue?,Imagine the Sun’s light is like a box of crayo...,None,Alright! Imagine the sky is like a big bowl of...,True,True,4.682507,1af9dae1-8130-46b1-9efd-2b9a381f2b0f,019a9efa-3393-71a5-80f1-eec579ee8f8e
2,How does string theory work?,"Imagina que todo lo que ves, incluso tú y yo, ...",None,"Okay! Imagine that everything in the universe,...",True,False,8.736447,4054da81-5869-484d-916c-e12c19059078,019a9efa-484d-70d1-9b1a-10772ba94e9c
3,How does photosynthesis work?,Imagina que una planta es como una cocinita má...,None,"Okay! Imagine plants are like tiny chefs, and ...",True,False,7.778148,8edadc77-fb59-4aa1-9bb3-4a0e369c50af,019a9efa-6dfd-7269-aebf-4e27b9b4b710
4,What is trustcall library?,Imagina que tienes un gran libro de dibujos y ...,None,"Alright, imagine you have a toy box where each...",True,True,6.479814,a661d0ce-9672-4a25-b813-104ea367db23,019a9efa-9048-7794-88fa-5b4c99d46d7d
5,How does a democracy work?,Imagina que tú y tus amigos quieren escoger un...,None,Okay! Imagine you and your friends want to dec...,True,False,7.127248,acd456da-cd68-4842-bcfe-60049549e0eb,019a9efa-aca1-740e-b4d6-ebd6e79a809c
6,What is LangSmith by LangChain?,Imagine you built a little talking robot that ...,None,Okay! Imagine you have a big box of toys that ...,False,True,10.696698,b80f2c35-2d23-400d-9963-83cfdb4a0026,019a9efa-ccc5-745d-86ec-0d3c6981801b
7,What is the Langchain framework?,Imagina que quieres montar un castillo de LEGO...,None,Okay! Imagine you want to build a really cool ...,True,True,7.235491,bb1147ea-5e7a-4445-bba4-fcafb900d5dc,019a9efa-f96d-7300-9cd3-4c710edd6e79
8,What is sound?,Sound is what we hear when something “wiggles”...,None,Okay! Imagine you have a drum. When you hit it...,True,True,5.162881,bf115918-1d03-48c0-a4ef-a33ee567533f,019a9efb-1b8d-748c-9841-131e0a90c24a
9,What is LangGraph?,Imagina que quieres construir un castillo de L...,None,"Okay, imagine you have a big box of LEGO brick...",True,True,5.745662,d7a5d945-7662-45ab-9099-634e1544c56c,019a9efb-327b-775a-ae02-928dcea607e8
